In [1]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav

# download and load all models
preload_models()

No GPU being used. Careful, inference might be very slow!
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [1]:
# https://colab.research.google.com/drive/1dWWkZzvu7L9Bunq9zvD-W02RFUXoW-Pd?usp=sharing#scrollTo=cWuW8SdgcO_Q
from transformers import AutoProcessor, BarkModel
import torch, os
import scipy

device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")
# model = BarkModel.from_pretrained("suno/bark-small")
model = model.to(device)

voice_preset = "v2/en_speaker_5"

/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/zaibachkhoa/Documents/bark/venv/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torc

In [6]:
# Py_Bark
text_prompt = "Hanoi is nicest at the night"
text_prompt = f"♪ {text_prompt} ♪"
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("Py_Bark.wav", SAMPLE_RATE, audio_array)

100%|███████████████████████████████████████████| 19/19 [02:00<00:00,  6.36s/it]


In [10]:
# Py_Transformers
inputs = processor(
    """
         ♪ Hanoi is nicest at the night ♪
    """,
    # voice_preset=voice_preset
)
# inputs = processor("[clears throat] Hello uh ..., my dog is cute [laughter]")
# # https://sunoaiwiki.com/en/resources/2024-05-13-list-of-metatags/

audio_array = model.generate(
    **inputs.to(device),
    length_penalty=2.2,
    num_beams=2,
    # # temperature=0.5,
    # # semantic_temperature=0.8,
)
audio_array = audio_array.cpu().numpy().squeeze()

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("Py_Transformers.wav", rate=sample_rate, data=audio_array)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
